In [8]:
import xarray as xr
import numpy as np
import scipy as sc
import sklearn as skl
import skimage as ski
from scipy.linalg import norm
from scipy.spatial.distance import euclidean, jensenshannon, correlation
from scipy.stats import wasserstein_distance, ecdf


In [5]:
## https://gist.github.com/larsmans/3116927
_SQRT2 = np.sqrt(2)     # sqrt(2) with default precision np.float64


def hellinger1(p, q):
    return norm(np.sqrt(p) - np.sqrt(q)) / _SQRT2


def hellinger2(p, q):
    return euclidean(np.sqrt(p), np.sqrt(q)) / _SQRT2


def hellinger3(p, q):
    return np.sqrt(np.sum((np.sqrt(p) - np.sqrt(q)) ** 2)) / _SQRT2

In [7]:
ecdf(sample)

pdf = derivative(ecdf)

jensenshannon(p, q, base=None, *, axis=0, keepdims=False)
# probability distribution -> requires ecdf and pdf

wasserstein_distance(u_values, v_values, u_weights=None, v_weights=None)
# probability distribution -> requires ecdf and pdf

correlation(u, v, w=None, centered=True)
#correlation distance


SyntaxError: iterable argument unpacking follows keyword argument unpacking (1837290051.py, line 5)